In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gono-unitigs/cip_sr_gwas_filtered_unitigs.Rtab
/kaggle/input/gono-unitigs/cfx_sr_gwas_filtered_unitigs.Rtab
/kaggle/input/gono-unitigs/azm_sr_gwas_filtered_unitigs.Rtab
/kaggle/input/gono-unitigs/metadata.csv


In [5]:
import matplotlib.pyplot as plt

**Predicting AMR in Gonorrhoea**

In [7]:
df= pd.read_csv("/kaggle/input/gono-unitigs/metadata.csv").dropna()
print("df shape:", df.shape)
print(df.info())
df.head(10)

df shape: (1059, 31)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1059 entries, 1688 to 3684
Data columns (total 31 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Sample_ID       1059 non-null   object 
 1   Year            1059 non-null   float64
 2   Country         1059 non-null   object 
 3   Continent       1059 non-null   object 
 4   Beta.lactamase  1059 non-null   object 
 5   Azithromycin    1059 non-null   object 
 6   Ciprofloxacin   1059 non-null   object 
 7   Ceftriaxone     1059 non-null   object 
 8   Cefixime        1059 non-null   object 
 9   Tetracycline    1059 non-null   object 
 10  Penicillin      1059 non-null   object 
 11  NG_MAST         1059 non-null   object 
 12  Group           1059 non-null   int64  
 13  azm_mic         1059 non-null   float64
 14  cip_mic         1059 non-null   float64
 15  cro_mic         1059 non-null   float64
 16  cfx_mic         1059 non-null   float64
 17  tet_mic  

,Sample_ID,Year,Country,Continent,Beta.lactamase,Azithromycin,Ciprofloxacin,Ceftriaxone,Cefixime,Tetracycline,...,log2_cro_mic,log2_cfx_mic,log2_tet_mic,log2_pen_mic,azm_sr,cip_sr,cro_sr,cfx_sr,tet_sr,pen_sr
1688,10356_1#1,1998.0,Brasil,America,S,0.032,0.002,0.002,0.016,0.5,...,-8.965784,-5.965784,-1.000000,-3.000000,0.0,0.0,0.0,0.0,0.0,0.0
1689,10356_1#10,2008.0,Brasil,America,S,0.125,0.19,0.012,0.016,24,...,-6.380822,-5.965784,4.584963,-3.000000,0.0,0.0,0.0,0.0,1.0,0.0
1690,10356_1#11,2008.0,Chile,America,S,0.75,6,0.023,0.016,1,...,-5.442222,-5.965784,0.000000,0.000000,0.0,1.0,0.0,0.0,0.0,0.0
1691,10356_1#12,2008.0,Ecuador,America,R,0.064,0.006,0.006,0.016,12,...,-7.380822,-5.965784,3.584963,0.584963,0.0,0.0,0.0,0.0,1.0,1.0
1692,10356_1#15,1998.0,USA,America,S,0.125,32,0.004,0.016,1.5,...,-7.965784,-5.965784,0.584963,-1.000000,0.0,1.0,0.0,0.0,1.0,0.0
1693,10356_1#16,1998.0,USA,America,S,0.094,0.002,0.004,0.016,0.75,...,-7.965784,-5.965784,-0.415037,-1.395929,0.0,0.0,0.0,0.0,0.0,0.0
1694,10356_1#17,1998.0,USA,America,S,0.094,0.002,0.002,0.016,0.38,...,-8.965784,-5.965784,-1.395929,-2.395929,0.0,0.0,0.0,0.0,0.0,0.0
1695,10356_1#18,1998.0,USA,America,S,0.125,0.003,0.004,0.016,0.38,...,-7.965784,-5.965784,-1.395929,-2.000000,0.0,0.0,0.0,0.0,0.0,0.0
1696,10356_1#19,1998.0,USA,America,S,0.064,0.002,0.003,0.016,0.38,...,-8.380822,-5.965784,-1.395929,-2.000000,0.0,0.0,0.0,0.0,0.0,0.0
1697,10356_1#2,1998.0,Caribbean,America,R,0.023,0.002,0.002,0.016,0.38,...,-8.965784,-5.965784,-1.395929,3.584963,0.0,0.0,0.0,0.0,0.0,1.0


In [12]:
df.sort_values("Group", ascending= True)
df["Group"].mean()
df["Group"].std()

349.2427762852437

In [17]:
df.select_dtypes("number")

,Year,Group,azm_mic,cip_mic,cro_mic,cfx_mic,tet_mic,pen_mic,log2_azm_mic,log2_cip_mic,log2_cro_mic,log2_cfx_mic,log2_tet_mic,log2_pen_mic,azm_sr,cip_sr,cro_sr,cfx_sr,tet_sr,pen_sr
1688,1998.0,372,0.032,0.002,0.002,0.016,0.50,0.125,-4.965784,-8.965784,-8.965784,-5.965784,-1.000000,-3.000000,0.0,0.0,0.0,0.0,0.0,0.0
1689,2008.0,265,0.125,0.190,0.012,0.016,24.00,0.125,-3.000000,-2.395929,-6.380822,-5.965784,4.584963,-3.000000,0.0,0.0,0.0,0.0,1.0,0.0
1690,2008.0,266,0.750,6.000,0.023,0.016,1.00,1.000,-0.415037,2.584963,-5.442222,-5.965784,0.000000,0.000000,0.0,1.0,0.0,0.0,0.0,0.0
1691,2008.0,258,0.064,0.006,0.006,0.016,12.00,1.500,-3.965784,-7.380822,-7.380822,-5.965784,3.584963,0.584963,0.0,0.0,0.0,0.0,1.0,1.0
1692,1998.0,195,0.125,32.000,0.004,0.016,1.50,0.500,-3.000000,5.000000,-7.965784,-5.965784,0.584963,-1.000000,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3680,2015.0,847,0.250,0.004,0.004,0.008,0.25,0.500,-2.000000,-7.965784,-7.965784,-6.965784,-2.000000,-1.000000,0.0,0.0,0.0,0.0,0.0,0.0
3681,2015.0,1050,0.120,0.002,0.004,0.008,0.25,0.250,-3.058894,-8.965784,-7.965784,-6.965784,-2.000000,-2.000000,0.0,0.0,0.0,0.0,0.0,0.0
3682,2015.0,1032,0.500,0.004,0.008,0.016,1.00,0.500,-1.000000,-7.965784,-6.965784,-5.965784,0.000000,-1.000000,0.0,0.0,0.0,0.0,0.0,0.0
3683,2015.0,186,0.500,8.000,0.060,0.060,2.00,1.000,-1.000000,3.000000,-4.058894,-4.058894,1.000000,0.000000,0.0,1.0,0.0,0.0,1.0,0.0


In [18]:
df.select_dtypes("object")

,Sample_ID,Country,Continent,Beta.lactamase,Azithromycin,Ciprofloxacin,Ceftriaxone,Cefixime,Tetracycline,Penicillin,NG_MAST
1688,10356_1#1,Brasil,America,S,0.032,0.002,0.002,0.016,0.5,0.125,64
1689,10356_1#10,Brasil,America,S,0.125,0.19,0.012,0.016,24,0.125,3791
1690,10356_1#11,Chile,America,S,0.75,6,0.023,0.016,1,1,3795
1691,10356_1#12,Ecuador,America,R,0.064,0.006,0.006,0.016,12,1.5,3771
1692,10356_1#15,USA,America,S,0.125,32,0.004,0.016,1.5,0.5,270
...,...,...,...,...,...,...,...,...,...,...,...
3680,SRR5827366,New_Zealand,Oceania,0,0.25,0.004,0.004,0.008,0.25,0.5,4186
3681,SRR5827371,New_Zealand,Oceania,0,0.12,0.002,0.004,0.008,0.25,0.25,7803
3682,SRR5827372,New_Zealand,Oceania,0,0.5,0.004,0.008,0.016,1,0.5,13990
3683,SRR5827369,New_Zealand,Oceania,0,0.5,8,0.06,0.06,2,1,2400
